### Importing libraries, using TOKEN & USER_ID

In [10]:
import sqlalchemy
import pandas as pd
from sqlalchemy.orm import sessionmaker
import requests
import json
from datetime import datetime
import datetime
import sqlite3

USER_ID = '227dt3g18rrzyco2j7p2boy2ly'
TOKEN = 'BQC_BmsCxoTaAITQi_O0GaAhhuc3wZ2LVYEOQyV14l52mb9lNHnLmPJe4hsdrMmm-RdKZowPRdZaxLVYRH_BNR9NoMcuoRpOj3dNC3yNfORz2F-dKfiTxGn867b1upfn07RHx3OJXCo2A59wdNLZkE8Voa0Jv92-DS96rWxE'

##### Function to validate my data after being extracted and cleaned up

In [11]:
def data_validation(df:pd.DataFrame):
#is the datadrame empty?    
    if df.empty:
        print("No songs were listened in the last 24 hours")
        return False

#are primary keys ok?

    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("The primary key conditions has been corrupted")

#are null values at song_id
    
    if df.isnull().values.any():
        raise Exception("There is a Null Value in the played_at column")
    else:
        return True
    
        

In [12]:
headers = {
    "Accept":"application/json",
    "Content-Type":"application/json",
    "Authorization":"Bearer {token}".format(token=TOKEN)
}
    

In [13]:
r = requests.get("https://api.spotify.com/v1/me/player/recently-played",headers = headers)
response = r.json()

In [27]:
def extraction():
    my_song_list = []

    for song in response['items']:
        artist_id = song['track']['artists'][0]['id']
        artist_name = song['track']['artists'][0]['name']
        artist_link = song['track']['artists'][0]['external_urls']['spotify'] 
        album_id = song['track']['album']['id']
        album_name = song['track']['album']['name']
        album_link = song['track']['album']['external_urls']['spotify']
        song_id = song['track']['id']
        song_name = song['track']['name']
        song_link = song['track']['external_urls']['spotify']
        duration_ms = song['track']['duration_ms']
        popularity = song['track']['popularity']
        disc_number = song['track']['disc_number']
        played_at = song['played_at'].split(".")[0]
        
        song_dic = {'artist_id': artist_id,
                        'artist_name':artist_name,
                        'artist_link':artist_link,
                        'album_id':album_id,
                        'album_name':album_name,
                        'album_link':album_link,
                        'song_id':song_id,
                        'song_name':song_name,
                        'song_link':song_link,
                        'duration_ms':duration_ms,
                        'popularity':popularity,
                        'disc_number':disc_number,
                        'played_at':played_at
                        }
        
        my_song_list.append(song_dic)
        df = pd.DataFrame(my_song_list)
        
        return True



##### The Clean-Up

In [25]:
def clean_up(df:pd.DataFrame):
    #Re-ordering columns in my df
    df = df[["artist_id","artist_link","album_id","album_name","album_link","song_id","song_name","song_link","duration_ms","popularity","disc_number","played_at"]]
    #Creating two columns (date, time) by spliting the played_at column.
    df[['date','time']] = df['played_at'].str.split('T',expand=True)
    #Right now, played_at, date & time are objects, so we need to change these to timestamp.
    df['date'] = pd.to_datetime(df['date'])
    df['time'] = pd.to_timedelta(df['time'])
    df['played_at'] = pd.to_datetime(df['played_at'])
    df['played_at'] = df['played_at'].dt.tz_localize('US/Central')
    ## df['played_at'] = df['played_at'].dt.strftime("%d/%m/%y")
    return True


In [31]:
if extraction():
    print("The extraction ran succesfully")
else:
    print("something came up during the extraction, check TOKEN, USER_ID or the .json response")
if clean_up(df):
    print("The cleaned up ran succesfully")
else:
    print("something came up")
if data_validation(df):
    print("Data validation ran succesfully")
else:
    print("something came up")


The extraction ran succesfully
The cleaned up ran succesfully
Data validation ran succesfully
